## 라이엇 API를 활용한 데이터 추출

### 순서
1. 리그데이터로 (/lol/match/v4/matchlists/by-account/{encryptedAccountId}) accountid 추출

2. accoutid롤 gameid 추출

3. gameid를 추출 후 분석에 필요한 match data 뽑기

In [1]:
# 패키지 불러오기

import pickle # 리스트 안의 데이터프레임 형태 저장
import requests # api 요청
import json
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time

#### 리그데이터 추출

In [ ]:
# API key를 riot 홈페이지에서 신청.
# API의 요청 제한은 2분에 100개

# api 요청
api_key = "라이엇 api에서 요청한 api키"

api_url_grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
api_url_master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
# 그랜드마스터, 마스터 데이터 가져와서 데이터프레임으로 변환
r = requests.get(api_url_grandmaster)
league_df = pd.DataFrame(r.json())
r = requests.get(api_url_master)
league_df = pd.concat([league_df, pd.DataFrame(r.json())], axis=0)

# entries 데이터프레임으로 변환하여 추가
league_df.reset_index(inplace=True)
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T
league_df = pd.concat([league_df, league_entries_df], axis=1)

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
league_df.info()

#### 리그데이터를 활용해 accountid 추출

In [ ]:
# API 요청 제한 : 1초에 20, 2분에 100

league_df['account_id'] = np.nan # account_id 초기화
for i, summoner_id in enumerate(league_df['summonerId']):
    # 각 소환사의 SummonerId와 API Key를 포함한 url을 만들고, Summoner API에서 AccountId를 가져와 채워넣는다.
    api_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(5)
        r = requests.get(api_url)
    account_id = r.json()['accountId']
    league_df.iloc[i, -1] = account_id

league_df.to_csv('LeagueData.csv')


In [14]:
import pandas as pd
league_df =pd.read_csv("LeagueData_CGM.csv",index_col=0, encoding='cp949')
league_df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,account_id
0,CHALLENGER,NRUMektyZ14UL_KWjpleZj3fbXYAYf0JbPE-Ij9Ev7pcv8Q,모르겠어요,800,135,108,False,False,False,False,eLIKrDkL_Cedtbgip_nyBbECjfhtS815yrzK5gNO1Glf09...
1,CHALLENGER,GPdnNLG8w3fvS8taBCSebMbwgoXI1Vu9pnGzLdhBTlpqrS0,HnJaH,514,86,49,False,False,True,False,34TQMZVsvAKB7JkAQN63kglfnbHMT7dxKwrBQT58_RwM1J...
2,CHALLENGER,Cn_hfAVRXeUXYVh3FIWZD8KQldd0v--m0TB86DzicL5tgzI,못할 때도 있지,616,235,210,False,False,False,True,OFPIPl-kfu3nA0ElMm9Jrur8rx_MDsTIcqppQZgjMpHsGc...
3,CHALLENGER,9w0i6cROjCqZ2SaEn2Q4t-c42qioIpIaMNwn1p_wYnAJyg,PowderSn0w,697,202,169,False,False,False,False,rTmgtLm2MZ_M-2lZLRYlP4k5yEVehsLMaEQEG8HNWIgQ
4,CHALLENGER,ptzesqDYYLbau9ItJostF5HeMXTUpZPQ_QW5tTRapJN1AZU,zzzusss,573,82,65,False,False,True,False,MP_sIFVZYJGuLsiiJ8JXEEZZ91A3RhlE_fCZ1sTgG-WLqp4


#### accountid를 활용해 gameid 데이터 추출

In [ ]:
#Match info 데이터 받기 (gameId 추출을 위해)

season = str(13)

match_info_df = pd.DataFrame()
for account_id in league_df['account_id']:
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + account_id + \
                  '?season=' + season + '&api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(5)
        r = requests.get(api_url)
    match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])
match_info_df.to_csv('MatchInfoData.csv')

In [16]:
# 저장된 데이터 받아오기
match_info_df = pd.read_csv("MatchInfoData_CGM.csv", index_col=0)
match_info_df.reset_index(inplace=True)
match_info_df.head()

,index,platformId,gameId,champion,queue,season,timestamp,role,lane
0,0,KR,4998756933,28,900,13,1.613160e+12,DUO_SUPPORT,NONE
1,1,KR,4998742438,497,420,13,1.613160e+12,DUO_SUPPORT,NONE
2,2,KR,4998681189,53,420,13,1.613150e+12,DUO_SUPPORT,BOTTOM
3,3,KR,4998645933,3,420,13,1.613150e+12,DUO_SUPPORT,BOTTOM
4,4,KR,4998601091,147,420,13,1.613150e+12,DUO_SUPPORT,BOTTOM


#### gameid를 추출 후 분석에 필요한 match data 뽑기

In [ ]:
# Match 데이터 받기 (gameId를 통해 경기의 승패, 팀원과 같은 정보가 담겨있다.)

# match_info_df = match_info_df.drop_duplicates('gameId')

match_df = pd.DataFrame()
for game_id in match_info_df['gameId']: # 이전의 매치에 대한 정보 데이터에서 게임 아이디를 가져온다
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matches/' + str(game_id) + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(5)
        r = requests.get(api_url)
    r_json = r.json()
    temp_df = pd.DataFrame(list(r_json.values()), index=list(r_json.keys())).T # 게임 아이디에 대한 매치 데이터를 받아서 추가
    match_df = pd.concat([match_df, temp_df])


match_df = pd.read_csv('MatchDataS.csv', index_col=0)

# 솔로 랭크와 팀 랭크 게임만 분석.
select_indices = (match_df['queue']==420) | (match_df['queue']==440)
match_df = match_df.loc[select_indices, :].reset_index(drop=True)

# DataFrame 내의 리스트들이 파일로 저장되었다가 불러지는 과정에서 문자로 인식됨
for column in ['teams', 'participants', 'participantIdentities']:
    match_df[column] = match_df[column].map(lambda v: eval(v)) # 각 값에 대해 eval 함수를 적용

match_df.to_csv('MatchData.csv') # 파일로 저장

In [18]:
match_df = pd.read_csv('MatchData.csv', index_col=0)
match_df.head()

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4998803925,48,440,13,1.613170e+12,NONE,JUNGLE
1,KR,4998822343,64,440,13,1.613170e+12,NONE,JUNGLE
2,KR,4998771752,876,440,13,1.613170e+12,NONE,JUNGLE
3,KR,4995772805,67,450,13,1.613070e+12,SOLO,BOTTOM
4,KR,4995688518,429,450,13,1.613060e+12,DUO,TOP


In [ ]:
## 매치 데이터에서 teams, participants, participantIdentities가 최종적으로 원하는 데이터이다.

match_teams_df = pd.DataFrame()
for i in range(len(match_df)):
    temp_df = pd.DataFrame(match_df['teams'].iloc[i]) # teams 데이터를 2행 짜리 데이터프레임으로 변환
    temp_df['gameId'] = match_df['gameId'].iloc[i] # teams 데이터에 각 게임의 gameId 추가 (2행 마다 같은 값)
    # teams 데이터에 있는 bans 데이터를 5개의 변수로 추가한다
    ban_dict = {i: pd.DataFrame(temp_df['bans'][i]).iloc[:, 0] for i in range(2)} # 각 팀의 밴픽을 저장
    temp_ban = pd.DataFrame(ban_dict).T
    temp_ban.columns = [f'ban{i}' for i in range(1, 6)] # 열 이름 변경
    temp_df = pd.concat([temp_df, temp_ban], axis=1)

    match_teams_df = pd.concat([match_teams_df, temp_df])

match_teams_df.to_csv('MatchTeamsData.csv')

In [22]:
match_teams_df = pd.read_csv('MatchTeamsData.csv', index_col=0).reset_index()
match_teams_df.drop('bans', axis=1, inplace=True)
match_teams_df.head()

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,4998756933,KR,1.613160e+12,796,900,11,13,11.3.357.5376,URF,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4998742438,KR,1.613160e+12,997,420,11,13,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4998681189,KR,1.613150e+12,1312,420,11,13,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4998645933,KR,1.613150e+12,1408,420,11,13,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4998601091,KR,1.613150e+12,1441,420,11,13,11.3.357.5376,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


#### pysql을 활용하여 db에 데이터 삽입

In [ ]:
import numpy as np
import pandas as pd
import pymysql
#이게 db넣는 코드
# T100 : Team 100 (Blue team) 챔피언 리스트
# T200 : Team 200 (Red team) 챔피언 리스트
# win : 100이면 Team 100 승리, 200이면 Team 200 승리
conn = pymysql.connect(host='localhost', user='root', password='dlaghwls21@', db='lolProject', charset='utf8')

curs = conn.cursor()

#이게 csv 
df = pd.read_csv('C:/Users/dlagh/lastdata.csv',index_col=0, encoding='utf-8')

for i in range(len(df)):
    sql1 = "insert into df_game(T100,T200,win) values ('"+df['T100'][i]+"','"+df['T200'][i]+"','"+str(df['win'][i])+"');"
    curs.execute(sql1)
    conn.commit() 
conn.close()